# Programming for Chemistry 2025/2026 @ UniMI

![logo](logo_small.png "Logo")

## Lecture 15: Applications to chemistry

In this lecture we'll use combine *linar algebra* and the `formula_to_dict()` function we created in lecture 6 to balance chemical equations. 

## 1. Chemical equations as linear systems

**TL;DR** ...balancing a chemical equation is equaivalent to solving a system of equations.

To make it simple let’s start with a simple chemical reaction:

$CH_4 + O_2 \rightarrow CO_2 + H_2O$

We are trying to determine the values of the coefficients in front of each compound so to start let’s give each coefficient a letter value of $x_0, x_1, x_2, x_3$:

$x_0 CH_4 + x_1 O_2 \rightarrow x_2 CO_2 + x_3 H_2O$

First, let's bring the *r.h.s.* to the *l.h.s.*:

$x_0 CH_4 + x_1 O_2 - x_2 CO_2 - x_3 H_2O \rightarrow 0$

From looking at the equation, it can be seen that there are only three elements: C, H and O. Let's write a equation to balance each element:

| element | equation |
| --- | --- |
| C | x0 - x2 = 0 |
| H | 4 x0 - 2 x3 = 0 | 
| O | 2 x1 - 2 x2 - x3 = 0 |

In matrix form this is $A x = 0$, where $x = [x_0, x_1, x_2, x_3]$ and:
\begin{equation}
A=\begin{pmatrix}
   1 &  0 & -1 &  0 \\
   4 &  0 &  0 & -2 \\
   0 &  2 & -2 & -1 \\
\end{pmatrix}
\end{equation}

Thus the chemical equation has been turned into a system of **(3 equations, 4 unknowns)** i.e. **(number of elements, number of molecules)**.

This system of equations is **overdetermined** (i.e. there is an infinite number of solutions). To solve it, we can calculate the **null space** of $A$ which is the set of vectors $x$ that solve the equation $A x = 0$. The results is $x=[1,2,1,2]$, that is:

$1\cdot CH_4 + 2\cdot O_2 \rightarrow 1\cdot CO_2 + 2\cdot H_2O$

This may seem an overkill, since this chemical equation is sufficiently simple that can be balanced at first sight. Try to balance this one:

$MnS + As_2Cr_{10}O_{35} + H_2SO_4 \rightarrow HMnO_4 + AsH_3 + CrS_3O_{12} + H_2O$.

### 1.1 Import the needed packages...
... and let's form a list with reactants and products.

In [1]:
import numpy as np
import scipy
import math

from parse_chemical_formula import formula_to_dict

In [5]:
# Given a chemical equation in a string like "O2 + H2 = H2O", write a function
# that splits the string and returns a list of reactants and a list of products

def get_reactants_prodcuts(eq):
    lhs, rhs = eq.split('=')
    #reactants = [x.strip() for x in lhs.split('+')]
    #products = [x.strip() for x in rhs.split('+')]
    reactants = lhs.split('+')
    products = rhs.split('+')
    return reactants, products

In [6]:
equations = ['O2 + H2 = H2O', 'CH4+O2 =CO2 + H2O', 'MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O']
for eq in equations:
    reactants, products = get_reactants_prodcuts(eq)
    print(eq, "=>", reactants, products)

O2 + H2 = H2O => ['O2 ', ' H2 '] [' H2O']
CH4+O2 =CO2 + H2O => ['CH4', 'O2 '] ['CO2 ', ' H2O']
MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O => ['MnS ', ' As2Cr10O35 ', ' H2SO4 '] [' HMnO4 ', ' AsH3 ', ' CrS3O12 ', ' H2O']


### 1.2 Build the $A$ matrix and solve $Ax = 0$

In [8]:
# Write a function that build the A matrix, given the list of reactants
# and products. Use formula_to_dict to extract the chemical formula

def build_matrix(reactants, products, debug=False):
    # count number of molecules
    nmols = len(reactants) + len(products)
    
    # make a list of elements and count number of unique elements
    elements = []
    for m in reactants + products:
        f = formula_to_dict(m)
        for e in f.keys():
            if e not in elements:
                elements.append(e)
    nelements = len(elements)            
        
    # create matrix A using np.zeros((nelements,nmols))
    A = np.zeros((nelements,nmols))
    
    # fill the matrix with the coefficients of the reactants
    for j in range(len(reactants)):
        f = formula_to_dict(reactants[j])
        for i in range(len(elements)):
            if elements[i] in f:
                A[i,j] = f[elements[i]]
        
    
    # fill the matrix with the coefficients of the products
    for j in range(len(products)):
        f = formula_to_dict(products[j])
        for i in range(len(elements)):
            if elements[i] in f:
                A[i,j+len(reactants)] = -f[elements[i]]    
    return A


In [9]:
equations = ['O2 + H2 = H2O', 'CH4+O2 =CO2+ H2O', 'MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O']
for eq in equations:
    reactants, products = get_reactants_prodcuts(eq)
    A = build_matrix(reactants, products, debug=True)
    print(eq)
    print(A)
    print()

O2 + H2 = H2O
[[ 2.  0. -1.]
 [ 0.  2. -2.]]

CH4+O2 =CO2+ H2O
[[ 1.  0. -1.  0.]
 [ 4.  0.  0. -2.]
 [ 0.  2. -2. -1.]]

MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O
[[  1.   0.   0.  -1.   0.   0.   0.]
 [  1.   0.   1.   0.   0.  -3.   0.]
 [  0.   2.   0.   0.  -1.   0.   0.]
 [  0.  10.   0.   0.   0.  -1.   0.]
 [  0.  35.   4.  -4.   0. -12.  -1.]
 [  0.   0.   2.  -1.  -3.   0.  -2.]]



In [12]:
# Write a function to solve the homogenenous linear system.
# 1. To get the the null space, use 'scipy.linalg.null_space(A)'.
# 2. If the null space has zero length, there is no solution.
#    If the length is >1, there are more solutions and let's think more about it.
# 3. If the length is 1, find the least common multiple that makes it a vector of integers.
# 4. Check that the integer vector is the correct solution)

def solve_linear_system(A):
    # get the null space
    x = scipy.linalg.null_space(A)
    
    # check number of solutions
    if x.shape[1] == 0:
        raise RuntimeError("the reaction cannot be balanced!")
    elif x.shape[1] > 1:
        raise RuntimeError("the reaction can be balanced in multiple ways!")
    
    # extract the vector, divide by the smallest value
    c = x[:,0]
    c = c / np.min(c)
    
    # find the l.c.m.
    for i in range(1,1000):
        cint = c * i
        if np.all( np.abs((np.round(cint) - cint)) < 1e-9 ):
            break
    
    # check if the solution is correct
    zero = A @ cint
    if not np.all( np.abs(zero) < 1e-9 ):
        raise RuntimeError("something is wrong")
        
    return cint


In [13]:
equations = ['O2 + H2 = H2O', 'CH4+O2 =CO2+ H2O', 'MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O']
for eq in equations:
    reactants, products = get_reactants_prodcuts(eq)
    A = build_matrix(reactants, products, debug=False)
    c = solve_linear_system(A)
    print(eq, c)

O2 + H2 = H2O [1. 2. 2.]
CH4+O2 =CO2+ H2O [1. 2. 1. 2.]
MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O [ 16.  13. 374.  16.  26. 130. 327.]


### 1.3 Wrap up the functions in a single one `balance_chemical_equation(eq)`...
... that returns the balanced equation as a string.

In [22]:
# write a function that calls get_reactants_products(), build_matrix() and
# solve_linear_system(), then returns the balanced reaction as a string
# like: "O2 + 2*H2 = 2*H2O"

def balance_chemical_reaction(eq):
    reactants, products = get_reactants_prodcuts(eq)
    A = build_matrix(reactants, products, debug=False)
    c = solve_linear_system(A)
    
    lhs = []
    for i in range(len(reactants)):
        lhs.append(str(round(c[i])) + '*' + reactants[i])

    rhs = []
    for i in range(len(products)):
        rhs.append(str(round(c[i+len(reactants)])) + '*' + products[i])
        
    return ' + '.join(lhs) + ' = ' + ' + '.join(rhs)        

In [23]:
equations = ['O2 + H2 = H2O', 'CH4+O2 =CO2+ H2O', 'MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O']
for eq in equations:
    balanced = balance_chemical_reaction(eq)
    print(eq, '  =>  ', balanced)

O2 + H2 = H2O   =>   1*O2  + 2* H2  = 2* H2O
CH4+O2 =CO2+ H2O   =>   1*CH4 + 2*O2  = 1*CO2 + 2* H2O
MnS + As2Cr10O35 + H2SO4 = HMnO4 + AsH3 + CrS3O12 + H2O   =>   16*MnS  + 13* As2Cr10O35  + 374* H2SO4  = 16* HMnO4  + 26* AsH3  + 130* CrS3O12  + 327* H2O


### 1.4 Let us try more chemical reactions...
... taken from a textbook. 

In [24]:
# this one is clearly wrong
eq = "CO + O2 = H2O"
print(balance_chemical_reaction(eq))

RuntimeError: the reaction cannot be balanced!

In [25]:
# this one cannot be balanced
eq = "H2O + NO2 = HNO3"
print(balance_chemical_reaction(eq))

RuntimeError: the reaction cannot be balanced!

In [26]:
# this one doesn't look complicated but...
eq = "KNO3 + S + C = K2CO3 + K2SO4 + CO2 + N2"
print(balance_chemical_reaction(eq))

# similar cases:
#eq = 'P + HNO3 + H2O = H3PO4 + NO + NO2'
#eq = 'MnO2 + SO2 = MnS2O6 + MnSO4'

RuntimeError: the reaction can be balanced in multiple ways!

In [27]:
equations = ['NaNO3 + Zn + NaOH = Na2ZnO2 + NH3 + H2O',
             'MgSO4 + NaOH = Mg(OH)2 + Na2SO4',
             'K4Fe(CN)6 + KMnO4 + H2SO4 = KHSO4 + Fe2(SO4)3 + MnSO4 + HNO3 + CO2 + H2O']

for eq in equations:
    balanced = balance_chemical_reaction(eq)
    print(eq, '  =>  ', balanced)

NaNO3 + Zn + NaOH = Na2ZnO2 + NH3 + H2O   =>   1*NaNO3  + 4* Zn  + 7* NaOH  = 4* Na2ZnO2  + 1* NH3  + 2* H2O
MgSO4 + NaOH = Mg(OH)2 + Na2SO4   =>   1*MgSO4  + 2* NaOH  = 1* Mg(OH)2  + 1* Na2SO4
K4Fe(CN)6 + KMnO4 + H2SO4 = KHSO4 + Fe2(SO4)3 + MnSO4 + HNO3 + CO2 + H2O   =>   10*K4Fe(CN)6  + 122* KMnO4  + 299* H2SO4  = 162* KHSO4  + 5* Fe2(SO4)3  + 122* MnSO4  + 60* HNO3  + 60* CO2  + 188* H2O


### 1.5 Let's balance ionic reactions as well
Let's extend our function to the case of ionic reactions. For instance:

$ Mo^{3+} + Ce^{4+} + H_2O \rightarrow MoO_4^{2-} + Ce^{3+} + H^+ $

In principle we should modify `parse_chemical_equation()` to extract the ionic charge for each molecule. To simplify our life, we will instead create a list of ionic charges for each molecule and pass it to a modified version of `build_matrix()` and `balance_chemical_equation()`. For the example above, `charges = [3, 4, 0, -2, 3, 1]`.

Note that to balance redox equations, we should also add an equation for the conservation of the charge. For the reaction above, the `A` matrix will be made of 4+1 rows and 6 columns *i.e. (number of atoms+1, number of molecules)*. The last row of the matrix `A` will be: `[3 4 0 2 -3 -1]`.

In [ ]:
# Write a function that build the A matrix, given the list of reactants
# and products. Use formula_to_dict to extract the chemical formula.
# If charges is not None, handle the case of ionic reactions.

def build_matrix(reactants, products, charges=None, debug=False):
    ...
    
    return A



In [ ]:
# write a function that calls get_reactants_products(), build_matrix() and
# solve_linear_system(), then returns the balanced reaction as a string
# like: "O2 + 2*H2 = 2*H2O". In the case of ionic reactions, print "^n"
# after each molecule, where n is che ionic charge

def balance_chemical_reaction(eq, charges=None):
    ...
    
    return ' + '.join(lhs) + ' = ' + ' + '.join(rhs)  

Let's try in on the following reactions:

* $ Mo^{3+} + Ce^{4+} + H_2O \rightarrow MoO_4^{2-} + Ce^{3+} + H^+ $
* $ MnO_4^- + Cl^- + H^+ \rightarrow Mn^{2+} + Cl2 + H_2O$

In [ ]:
eq = "Mo + Ce + H2O = MoO4 + Ce + H"
charges = [3, 4, 0, -2, 3, 1]

balanced = balance_chemical_reaction(eq, charges)
print(balanced)

In [ ]:
eq = "MnO4 + Cl + H = Mn + Cl2 + H2O"
charges = [-1, -1, 1, 2, 0, 0]

balanced = balance_chemical_reaction(eq, charges)
print(balanced)